# Chest X-Ray Classification with TensorFlow/Keras

This notebook demonstrates:
- How to load and split data using functions from `prepare_data.py`
- How to build a custom data generator to feed images to the model.
- How to define, train, and evaluate a simple CNN using TensorFlow/Keras.


1) Get Train / Val / Test Splits

In [1]:
# Section 1: Importing Libraries and Loading Data
import os
import numpy as np
import pandas as pd
from prepare_data import get_data, preprocess_image

# Load train, test, and validation DataFrames using the get_data function
train_data, test_data, val_data = get_data()

# Define a mapping from label strings to integers
label_map = {
    "No Finding": 0,
    "Effusion": 1,
    "Pneumothorax": 2,
    "Atelectasis": 3
}

# Path to the directory containing the images
IMAGES_DIR = "THE GIANT DATASET/CXR8/images"

print("Data loaded and paths set.")
print(f"Train samples: {len(train_data)}, Validation samples: {len(val_data)}, Test samples: {len(test_data)}")


Data loaded and paths set.
Train samples: 16800, Validation samples: 3600, Test samples: 3600


2) Defining Data Generator

In [2]:
import torch

def data_generator(data_df, images_dir, batch_size=8, shuffle=True):
    """
    Generator yielding batches of (images, labels) as PyTorch tensors.
    """
    num_samples = len(data_df)
    while True:
        if shuffle:
            data_df = data_df.sample(frac=1).reset_index(drop=True)
        for start_idx in range(0, num_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_samples)
            batch_df = data_df.iloc[start_idx:end_idx]

            batch_images = []
            batch_labels = []

            for _, row in batch_df.iterrows():
                img_name = row['Image Index']
                label_str = row['Finding Labels']
                label_int = label_map[label_str]

                img_path = os.path.join(images_dir, img_name)
                if not os.path.exists(img_path):
                    print(f"File not found: {img_path}")
                    continue  

                img_array = preprocess_image(img_path, target_size=(512, 512))
                img_array = np.transpose(img_array, (2, 0, 1))  # Convert to (C, H, W)

                batch_images.append(img_array)
                batch_labels.append(label_int)

            if batch_images and batch_labels:
                batch_x = torch.tensor(np.array(batch_images), dtype=torch.float32)  # Convert to tensor
                batch_y = torch.tensor(batch_labels, dtype=torch.long)  # Convert to tensor
                yield batch_x, batch_y  # Return PyTorch tensors


3) Defining the PyTorch Model

In [ ]:
# Section 3: Define PyTorch Model
import torch
import torch.nn as nn
import torch.optim as optim

class ChestXRayCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(ChestXRayCNN, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Convolutional Block 1
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # Convolutional Block 2
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Convolutional Block 3
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  # Convolutional Block 4
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),  # Convolutional Block 5
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 16 * 16, 512),  # Adjust input size based on input image size
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
model = ChestXRayCNN(num_classes=4).to(device)
print(model)


4) Training and Evaluation

In [3]:
# Import fresh resnet50 model
from torchvision import models
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

resnet50_model = models.resnet50(pretrained=True)
resnet50_model.fc = nn.Linear(resnet50_model.fc.in_features, 4)  # Modify output layer for 4 classes
resnet50_model = resnet50_model.to(device)
print(resnet50_model)

cuda


c:\Users\akbul\OneDrive\Desktop\Projects\KutGuard\Chest X-Ray Diagnosis\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\akbul\OneDrive\Desktop\Projects\KutGuard\Chest X-Ray Diagnosis\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
# Import fresh resnet18 model
from torchvision import models
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

resnet18_model = models.resnet18(pretrained=True)
resnet18_model.fc = nn.Linear(resnet18_model.fc.in_features, 4) # Modify output layer for 4 classes
resnet18_model = resnet18_model.to(device)
print(resnet18_model)

cuda


c:\Users\akbul\OneDrive\Desktop\Projects\KutGuard\Chest X-Ray Diagnosis\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\akbul\OneDrive\Desktop\Projects\KutGuard\Chest X-Ray Diagnosis\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\akbul/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 107MB/s] 


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Import fresh resnet152 model
from torchvision import models
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

resnet152_model = models.resnet152(pretrained=True)
resnet152_model.fc = nn.Linear(resnet152_model.fc.in_features, 4) # Modify output layer for 4 classes
resnet152_model = resnet152_model.to(device)
print(resnet152_model)

cuda


AttributeError: module 'torchvision.models' has no attribute 'resnet10'

In [ ]:
# Import local resnet50 model
from torchvision import models
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

resnet50_model = models.resnet50(pretrained=False)  # Initialize architecture
resnet50_model.fc = nn.Linear(resnet50_model.fc.in_features, 4)  # Modify output layer for 4 classes
resnet50_model.load_state_dict(torch.load("chest_xray_resnet50.pth"))  # Load weights
resnet50_model = resnet50_model.to(device)  # Move to GPU if available

In [16]:
from tqdm import tqdm  # Import tqdm
import torch.optim as optim


# Section 4: Training and Evaluation
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(resnet18_model.parameters(), lr=1e-4)  # ✅ Use AdamW
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2, verbose=True)  # ✅ Add LR scheduler


BATCH_SIZE = 10
EPOCHS = 5

# Compute steps per epoch based on dataset sizes
train_steps = len(train_data) // BATCH_SIZE
val_steps = len(val_data) // BATCH_SIZE
test_steps = len(test_data) // BATCH_SIZE

# Create PyTorch data loaders
train_loader = data_generator(train_data, IMAGES_DIR, batch_size=BATCH_SIZE, shuffle=True)
val_loader = data_generator(val_data, IMAGES_DIR, batch_size=BATCH_SIZE, shuffle=False)
test_loader = data_generator(test_data, IMAGES_DIR, batch_size=BATCH_SIZE, shuffle=False)

# **Training loop with live progress bar**
for epoch in range(EPOCHS):
    resnet18_model.train()
    running_loss = 0.0
    progress_bar = tqdm(range(train_steps), desc=f"Epoch {epoch+1}/{EPOCHS}")

    for _ in progress_bar:
        images, labels = next(train_loader)  # Get batch from generator
        images, labels = images.to(device), labels.to(device)  # Move to GPU

        optimizer.zero_grad()
        outputs = resnet18_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=f"{loss.item():.4f}")  # Live loss update

    avg_loss = running_loss / train_steps
    scheduler.step(avg_loss)  # ✅ Update LR based on validation loss

    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {avg_loss:.4f}")
    print(f"Current Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")  # ✅ Show LR update


# **Evaluation Loop with Live Progress**
resnet18_model.eval()
correct, total = 0, 0
progress_bar = tqdm(range(test_steps), desc="Evaluating")

with torch.no_grad():
    for _ in progress_bar:
        images, labels = next(test_loader)  # Get batch
        images, labels = images.to(device), labels.to(device)

        outputs = resnet18_model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Show live accuracy update
        accuracy = 100 * correct / total
        progress_bar.set_postfix(acc=f"{accuracy:.2f}%")

# **Final Accuracy**
print(f"Final Test Accuracy: {accuracy:.2f}%")


Epoch 1/5: 100%|██████████| 1680/1680 [08:10<00:00,  3.42it/s, loss=0.2418]


Epoch [1/5] Loss: 0.3118
Current Learning Rate: 0.000100


Epoch 2/5: 100%|██████████| 1680/1680 [08:09<00:00,  3.43it/s, loss=0.0534]


Epoch [2/5] Loss: 0.2047
Current Learning Rate: 0.000100


Epoch 3/5: 100%|██████████| 1680/1680 [08:10<00:00,  3.42it/s, loss=0.2389]


...

Epoch [4/5] Loss: 0.1286
Current Learning Rate: 0.000100


Epoch 5/5: 100%|██████████| 1680/1680 [08:15<00:00,  3.39it/s, loss=0.1089]


Epoch [5/5] Loss: 0.1149
Current Learning Rate: 0.000100


Evaluating: 100%|██████████| 360/360 [01:38<00:00,  3.67it/s, acc=67.64%]

Final Test Accuracy: 67.64%


In [17]:
# Evaluate with training data
# **Evaluation Loop with Live Progress**

resnet18_model.eval()
correct, total = 0, 0
progress_bar = tqdm(range(train_steps), desc="Evaluating")

with torch.no_grad():
    for _ in progress_bar:
        images, labels = next(train_loader)  # Get batch
        images, labels = images.to(device), labels.to(device)

        outputs = resnet18_model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Show live accuracy update
        accuracy = 100 * correct / total
        progress_bar.set_postfix(acc=f"{accuracy:.2f}%")

# **Final Accuracy**
print(f"Final Test Accuracy: {accuracy:.2f}%")

Evaluating:   3%|▎         | 45/1680 [00:11<06:53,  3.95it/s, acc=96.22%]


KeyboardInterrupt: 

5) Save the Model

In [18]:
torch.save(resnet50_model.state_dict(), "chest_xray_resnet18.pth")

## Conclusion

- The model was built and trained using TensorFlow/Keras.
- Data was loaded and preprocessed on-the-fly using a custom generator.
- We evaluated the model on a separate test set.

**Next Steps:**
- Experiment with additional data augmentation.
- Consider adjusting the model architecture or training parameters.
- Explore deeper networks or transfer learning approaches if needed.
